# L3: Agentic Sales Pipeline

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>

## Initial Imports

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from helper import load_env
load_env()

import os
import yaml
from crewai import Agent, Task, Crew

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

## Load API tokens for our 3rd party APIs

In [2]:
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'

## Loading Tasks and Agents YAML files

In [3]:
# Define file paths for YAML configurations
files = {
    'lead_agents': 'config/lead_qualification_agents.yaml',
    'lead_tasks': 'config/lead_qualification_tasks.yaml',
    'email_agents': 'config/email_engagement_agents.yaml',
    'email_tasks': 'config/email_engagement_tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
lead_agents_config = configs['lead_agents']
lead_tasks_config = configs['lead_tasks']
email_agents_config = configs['email_agents']
email_tasks_config = configs['email_tasks']

## Create Pydantic Models for Structured Output

In [4]:
from pydantic import BaseModel, Field
from typing import Dict, Optional, List, Set, Tuple

class LeadPersonalInfo(BaseModel):
    name: str = Field(..., description="The full name of the lead.")
    job_title: str = Field(..., description="The job title of the lead.")
    role_relevance: int = Field(..., ge=0, le=10, description="A score representing how relevant the lead's role is to the decision-making process (0-10).")
    professional_background: Optional[str] = Field(..., description="A brief description of the lead's professional background.")

class CompanyInfo(BaseModel):
    company_name: str = Field(..., description="The name of the company the lead works for.")
    industry: str = Field(..., description="The industry in which the company operates.")
    company_size: int = Field(..., description="The size of the company in terms of employee count.")
    revenue: Optional[float] = Field(None, description="The annual revenue of the company, if available.")
    market_presence: int = Field(..., ge=0, le=10, description="A score representing the company's market presence (0-10).")

class LeadScore(BaseModel):
    score: int = Field(..., ge=0, le=100, description="The final score assigned to the lead (0-100).")
    scoring_criteria: List[str] = Field(..., description="The criteria used to determine the lead's score.")
    validation_notes: Optional[str] = Field(None, description="Any notes regarding the validation of the lead score.")

class LeadScoringResult(BaseModel):
    personal_info: LeadPersonalInfo = Field(..., description="Personal information about the lead.")
    company_info: CompanyInfo = Field(..., description="Information about the lead's company.")
    lead_score: LeadScore = Field(..., description="The calculated score and related information for the lead.")

## Importing Tools

In [5]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

## Lead Qualification Crew, Agents and Tasks

In [6]:
# Creating Agents
lead_data_agent = Agent(
  config=lead_agents_config['lead_data_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()]
)

cultural_fit_agent = Agent(
  config=lead_agents_config['cultural_fit_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()]
)

scoring_validation_agent = Agent(
  config=lead_agents_config['scoring_validation_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()]
)

# Creating Tasks
lead_data_task = Task(
  config=lead_tasks_config['lead_data_collection'],
  agent=lead_data_agent
)

cultural_fit_task = Task(
  config=lead_tasks_config['cultural_fit_analysis'],
  agent=cultural_fit_agent
)

scoring_validation_task = Task(
  config=lead_tasks_config['lead_scoring_and_validation'],
  agent=scoring_validation_agent,
  context=[lead_data_task, cultural_fit_task],
  output_pydantic=LeadScoringResult
)

# Creating Crew
lead_scoring_crew = Crew(
  agents=[
    lead_data_agent,
    cultural_fit_agent,
    scoring_validation_agent
  ],
  tasks=[
    lead_data_task,
    cultural_fit_task,
    scoring_validation_task
  ],
  verbose=True
)

## Email Engagement Crew

In [7]:
# Creating Agents
email_content_specialist = Agent(
  config=email_agents_config['email_content_specialist']
)

engagement_strategist = Agent(
  config=email_agents_config['engagement_strategist']
)

# Creating Tasks
email_drafting = Task(
  config=email_tasks_config['email_drafting'],
  agent=email_content_specialist
)

engagement_optimization = Task(
  config=email_tasks_config['engagement_optimization'],
  agent=engagement_strategist
)

# Creating Crew
email_writing_crew = Crew(
  agents=[
    email_content_specialist,
    engagement_strategist
  ],
  tasks=[
    email_drafting,
    engagement_optimization
  ],
  verbose=True
)

2025-08-24 20:05:28,428 - 140362482531200 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


## Creating Complete Sales Flow

In [16]:
from crewai import Flow
from crewai.flow.flow import listen, start

class SalesPipeline(Flow):
    @start()
    def fetch_leads(self):
        # Pull our leads from the database
        leads = [
            {
                "lead_data": {
                    "name": "Girish Gupta",
                    "job_title": "ML Enginnering Lead",
                    "company": "Delivery Hero SE",
                    "email": "girish.guptae@dh.com",
                    "use_case": "Using AI Agent to do better data enrichment."
                },
            },
        ]
        return leads

    @listen(fetch_leads)
    def score_leads(self, leads):
        scores = lead_scoring_crew.kickoff_for_each(leads)
        self.state["score_crews_results"] = scores
        return scores

    @listen(score_leads)
    def store_leads_score(self, scores):
        # Here we would store the scores in the database
        return scores

    @listen(score_leads)
    def filter_leads(self, scores):
        return [score for score in scores if score['lead_score'].score > 70]

    @listen(filter_leads)
    def write_email(self, leads):
        scored_leads = [lead.to_dict() for lead in leads]
        emails = email_writing_crew.kickoff_for_each(scored_leads)
        return emails

    @listen(write_email)
    def send_email(self, emails):
        # Here we would send the emails to the leads
        return emails

flow = SalesPipeline()

## Plotting the Flow

In [17]:
flow.plot()

Graph saved as crewai_flow_graph.html


In [18]:
from IPython.display import IFrame

IFrame(src='./crewai_flow.html', width='150%', height=600)

## Flow Kickoff

In [19]:
emails = await flow.kickoff()

2025-08-24 20:11:28,714 - 140362482531200 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Lead Data Specialist
## Task: Collect and analyze the following information about the lead:
- Personal Information:
  - Name: Obtain the full name of the lead.
  - Job Title: Determine the lead's current job title.
  - Role Relevance: Assess how relevant the lead's role is to the decision-making process on a scale from 0 to 10.
  - Professional Background: Optionally, gather a brief description of the lead's professional background.

- Company Information:
  - Company Name: Identify the name of the company the lead works for.
  - Industry: Determine the industry in which the company operates.
  - Company Size: Estimate the size of the company in terms of employee count.
  - Revenue: If available, collect information on the annual revenue of the company.
  - Market Presence: Evaluate the company's market presence on a scale from 0 to 10.

- Our Company and Product:
  - Company Name: CrewAI
  - Product: Multi-Agent Orchestration Platform
  - ICP: Enterprise companies looking int



# Agent: Lead Data Specialist
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"Delivery Hero SE company information industry size revenue market presence\"}"
## Tool Output: 

Search results: Title: Delivery Hero - Wikipedia
Link: https://en.wikipedia.org/wiki/Delivery_Hero
Snippet: Delivery Hero SE is a German multinational online food ordering and food delivery company based in Berlin, Germany. Founded in 2011, the company operates in ...
---
Title: Delivery Hero SE Company Profile - GlobalData
Link: https://www.globaldata.com/company-profile/delivery-hero-se/
Snippet: Missing: size presence
---
Title: Delivery Hero
Link: https://ir.deliveryhero.com/
Snippet: Approx. 90% of the Group's GMV is generated from markets where we hold the #1 position.1 Our footprint reaches over 2 billion people globally.
---
Title: Delivery Hero revenue 2013-2024 - Statista
Link: https://www.statista.com/statistics/696451/delivery-hero-revenue/
Snippet: In 2024, the revenue of De



# Agent: Lead Data Specialist
## Final Answer: 
### Comprehensive Data Report

#### Personal Information:
- **Name:** Girish Gupta
- **Job Title:** ML Engineering Lead
- **Role Relevance:** 8/10 (As an ML Engineering Lead, Girish plays a significant role in implementing AI solutions that could influence decision-making related to agentic automation.)
- **Professional Background:** 
  - Girish is an analytics leader and consultant with over 15 years of blended experience in Machine Learning and data analytics. He has led various key projects focused on data-driven insights and operational improvements. His experience includes teaching advanced analytical methods and managing teams, showcasing a strong blend of technical and leadership capabilities.

#### Company Information:
- **Company Name:** Delivery Hero SE
- **Industry:** Online Food Ordering and Delivery
- **Company Size:** 47,981 employees (as of 2023)
- **Revenue:** €12.8 billion (approximately for the year 2024)
- **Market Pr



# Agent: Cultural Fit Analyst
## Thought: Thought: It seems that the website requires JavaScript and cookies to display its content. I will now read the content from a different link that may be more accessible for gathering the information I need.
## Using tool: Read website content
## Tool Input: 
"{\"website_url\": \"https://www.deliveryhero.com/blog/delivery-hero-herocommunities/\"}"
## Tool Output: 
Just a moment...Enable JavaScript and cookies to continue


You ONLY have access to the following tools, and should NEVER make up tools that are not listed here:

Tool Name: Search the internet(**kwargs: Any) -> Any
Tool Description: Search the internet(search_query: 'string') - A tool that can be used to search the internet with a search_query. search_query: 'Mandatory search query you want to use to search the internet'
Tool Arguments: {'search_query': {'description': 'Mandatory search query you want to use to search the internet', 'title': 'Search Query', 'type': 'string'}}
Tool N



# Agent: Cultural Fit Analyst
## Thought: Thought: I have found relevant links discussing Delivery Hero SE’s core values and mission. The link regarding their mission from CanvasBusinessModel.com seems to provide a good overview.
## Using tool: Read website content
## Tool Input: 
"{\"website_url\": \"https://canvasbusinessmodel.com/blogs/mission/delivery-hero-mission\"}"
## Tool Output: 
What are Mission Vision & Core Values of Delivery Hero Company? – CanvasBusinessModel.com
Skip to content
Framework Templates
Blogs
Search
Cart
0
/
$0.00
 EN
EN
ES
FR
PT
What Are Delivery Hero's Mission, Vision & Core Values?July 11, 2025
DELIVERY HERO BUNDLE
 Get the Full Package:
5 Forces
$15
$10
BCG Matrix
$15
$10
Canvas
$15
$10
Marketing Mix
$15
$10
PESTLE
$15
$10
SWOT Analysis
$15
$10
TOTAL: ADD TO CART 
Decoding Delivery Hero: What Drives Its Success?
Every successful company is built on a foundation of purpose. Understanding the Delivery Hero Canvas Business Model, alongside its mission, visi



# Agent: Cultural Fit Analyst
## Final Answer: 
**
The assessment indicates a strong cultural fit score of **8/10**, based on aligned values regarding innovation, customer-centric approaches, and employee engagement initiatives, reflecting a positive potential partnership between CrewAI and Delivery Hero.


# Agent: Lead Scorer and Validator
## Task: Aggregate the collected data and perform the following steps: - Score Calculation: Based on predefined criteria, calculate a final lead score (0-100). Consider factors such as:
  - Role Relevance
  - Company Size
  - Market Presence
  - Cultural Fit
- Scoring Criteria Documentation: List the criteria used to determine the score. - Validation: Review the collected data and the calculated score for consistency and accuracy. Make adjustments if necessary. - Final Report: Compile a summary report that includes the final validated lead score, the criteria used, and any validation notes.
- Our Company and Product:
  - Company Name: CrewAI
  - 



# Agent: Lead Scorer and Validator
## Final Answer: 
---
## Lead Score Report

### Final Lead Score
**Score:** 90/100

### Scoring Criteria
- **Role Relevance (Max: 25):** 25
   - Justification: Girish Gupta's position as ML Engineering Lead directly aligns with CrewAI's focus on AI automation.
- **Company Size (Max: 25):** 20
   - Justification: Delivery Hero's large size (47,981 employees) indicates capability but also operational complexity.
- **Market Presence (Max: 25):** 25
   - Justification: Operates in over 70 countries, a significant factor for market reach and scalability.
- **Cultural Fit (Max: 25):** 20
   - Justification: Strong alignment with a tech-driven culture but potential concerns over gig economy practices.

### Validation Notes
The scores were assigned based on available data on Girish Gupta's role and the operational scope of Delivery Hero SE. The lead scoring process was verified through expansive company research outlining size, market presence, and cultural

2025-08-24 20:13:05,555 - 140362482531200 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Email Content Writer
## Task: Craft a highly personalized email using the lead's name, job title, company information, and any relevant personal or company achievements. The email should speak directly to the lead's interests and the needs of their company. This is not as cold outreach as it is a follow up to a lead form, so keep it short and to the point. Don't use any salutations or closing remarks, nor too complex sentences.
Our Company and Product: - Company Name: CrewAI - Product: Multi-Agent Orchestration Platform - ICP: Enterprise companies looking into Agentic automation. - Pitch: We are a platform that allows you to orchestrate AI Agents for automations to any vertical.
Use the following information: Personal Info: {'name': 'Girish Gupta', 'job_title': 'ML Engineering Lead', 'role_relevance': 10, 'professional_background': None} Company Info: {'company_name': 'Delivery Hero SE', 'industry': 'Food Delivery', 'company_size': 47981, 'revenue': None, 'market_presence': 10

## Usage Metrics and Costs

Let’s see how much it would cost each time if this crew runs at scale.

In [20]:
import pandas as pd

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([flow.state["score_crews_results"][0].token_usage.dict()])

# Calculate total costs
costs = 0.150 * df_usage_metrics['total_tokens'].sum() / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Display the DataFrame
df_usage_metrics

Total costs: $0.0222


,total_tokens,prompt_tokens,completion_tokens,successful_requests
0,147870,140337,7533,44


In [21]:
import pandas as pd

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([emails[0].token_usage.dict()])

# Calculate total costs
costs = 0.150 * df_usage_metrics['total_tokens'].sum() / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Display the DataFrame
df_usage_metrics

Total costs: $0.0003


,total_tokens,prompt_tokens,completion_tokens,successful_requests
0,1858,1457,401,3


## Inspecting Results

In [22]:
scores = flow.state["score_crews_results"]

In [23]:
import pandas as pd
from IPython.display import display, HTML

lead_scoring_result = scores[0].pydantic

# Create a dictionary with the nested structure flattened
data = {
    'Name': lead_scoring_result.personal_info.name,
    'Job Title': lead_scoring_result.personal_info.job_title,
    'Role Relevance': lead_scoring_result.personal_info.role_relevance,
    'Professional Background': lead_scoring_result.personal_info.professional_background,
    'Company Name': lead_scoring_result.company_info.company_name,
    'Industry': lead_scoring_result.company_info.industry,
    'Company Size': lead_scoring_result.company_info.company_size,
    'Revenue': lead_scoring_result.company_info.revenue,
    'Market Presence': lead_scoring_result.company_info.market_presence,
    'Lead Score': lead_scoring_result.lead_score.score,
    'Scoring Criteria': ', '.join(lead_scoring_result.lead_score.scoring_criteria),
    'Validation Notes': lead_scoring_result.lead_score.validation_notes
}

# Convert the dictionary to a DataFrame
df = pd.DataFrame.from_dict(data, orient='index', columns=['Value'])

# Reset the index to turn the original column names into a regular column
df = df.reset_index()

# Rename the index column to 'Attribute'
df = df.rename(columns={'index': 'Attribute'})

# Create HTML table with bold attributes and left-aligned values
html_table = df.style.set_properties(**{'text-align': 'left'}) \
                     .format({'Attribute': lambda x: f'<b>{x}</b>'}) \
                     .hide(axis='index') \
                     .to_html()

# Display the styled HTML table
display(HTML(html_table))

Attribute,Value
Name,Girish Gupta
Job Title,ML Engineering Lead
Role Relevance,10
Professional Background,None
Company Name,Delivery Hero SE
Industry,Food Delivery
Company Size,47981
Revenue,None
Market Presence,10
Lead Score,90


## Results

In [24]:
import textwrap

result_text = emails[0].raw
wrapped_text = textwrap.fill(result_text, width=80)
print(wrapped_text)

Hi Girish, as the ML Engineering Lead at Delivery Hero SE, you’re at the
forefront of innovation in the food delivery industry. Our Multi-Agent
Orchestration Platform at CrewAI can significantly enhance your automation
processes. Imagine orchestrating AI agents tailored for your operational needs
within your delivery network.   Ready to boost efficiencies and streamline
workflows? Schedule a demo with us today—click here [insert link] to choose a
time that works for you!   Interested in seeing immediate benefits? Check out
our case studies [insert link] to discover how similar companies are thriving
with our platform.   Take the next step towards a more efficient future—let's
get started today!


## How Complex Can it Get?

In [25]:
from crewai import Flow
from crewai.flow.flow import listen, start, and_, or_, router

class SalesPipeline(Flow):
    
  @start()
  def fetch_leads(self):
    # Pull our leads from the database
    # This is a mock, in a real-world scenario, this is where you would
    # fetch leads from a database
    leads = [
      {
        "lead_data": {
          "name": "João Moura",
          "job_title": "Director of Engineering",
          "company": "Clearbit",
          "email": "joao@clearbit.com",
          "use_case": "Using AI Agent to do better data enrichment."
        },
      },
    ]
    return leads

  @listen(fetch_leads)
  def score_leads(self, leads):
    scores = lead_scoring_crew.kickoff_for_each(leads)
    self.state["score_crews_results"] = scores
    return scores

  @listen(score_leads)
  def store_leads_score(self, scores):
    # Here we would store the scores in the database
    return scores

  @listen(score_leads)
  def filter_leads(self, scores):
    return [score for score in scores if score['lead_score'].score > 70]

  @listen(and_(filter_leads, store_leads_score))
  def log_leads(self, leads):
    print(f"Leads: {leads}")

  @router(filter_leads, paths=["high", "medium", "low"])
  def count_leads(self, scores):
    if len(scores) > 10:
      return 'high'
    elif len(scores) > 5:
      return 'medium'
    else:
      return 'low'

  @listen('high')
  def store_in_salesforce(self, leads):
    return leads

  @listen('medium')
  def send_to_sales_team(self, leads):
    return leads

  @listen('low')
  def write_email(self, leads):
    scored_leads = [lead.to_dict() for lead in leads]
    emails = email_writing_crew.kickoff_for_each(scored_leads)
    return emails

  @listen(write_email)
  def send_email(self, emails):
    # Here we would send the emails to the leads
    return emails

## Plotting the Flow

In [26]:
flow = SalesPipeline()
flow.plot()

Graph saved as crewai_flow_graph.html


In [27]:
from IPython.display import IFrame

IFrame(src='./crewai_flow_complex.html', width='150%', height=600)